# Imports and initialization

In [1]:
import requests
from pprint import pprint
from datetime import datetime, timezone

sgd_amount = 3000

now = datetime.now(tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')

ratios = {'utc_time': now}

# Get current conversion from MasterCard

In [2]:
url = 'https://www.mastercard.com/settlement/currencyrate/conversion-rate'
params = {'fxDate':'0000-00-00', # current rate
          'transCurr': 'SGD', 
          'crdhldBillCurr': 'MYR', 
          'bankFee': 0, 
          'transAmt': sgd_amount}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:140.0) Gecko/20100101 Firefox/140.0'}
response = requests.get(url=url, headers=headers, params=params)

print(response.json()['date'])
result = response.json()['data']
pprint(result)

ratio = result['conversionRate']
# alternatively:
# ratio = result['crdhldBillAmt'] / result['transAmt']

assert ratio > 3 and ratio < 3.6

ratios['MASTERCARD'] = ratio

2025-06-27 17:18:26
{'conversionRate': 3.3349064,
 'crdhldBillAmt': 10004.7192,
 'crdhldBillCurr': 'MYR',
 'fxDate': '2025-06-26',
 'transAmt': 3000,
 'transCurr': 'SGD'}


# Get current conversion from Wise

In [3]:
url = 'https://api.wise.com/v3/quotes/'
params = {'sourceCurrency': 'MYR', 
          'targetCurrency': 'SGD', 
          'sourceAmount': 'null', 
          'targetAmount': sgd_amount }

x = requests.post(url, json = params, headers=headers)

results = [i for i in x.json()['paymentOptions'] if i['payIn'] in ['BALANCE', 'BANK_TRANSFER']]

# results = {'WISE_FROM_'+r['payIn']: {r['sourceCurrency']: r['sourceAmount'], r['targetCurrency']:r['targetAmount']} for r in results}
results = {'WISE_FROM_'+r['payIn']: r['sourceAmount'] / r['targetAmount'] for r in results}
for r in results.values():
    assert r > 3 and r < 3.6

ratios.update(results)

# Store data

In [4]:
print(ratios)

{'utc_time': '2025-06-27 17:18:25', 'MASTERCARD': 3.3349064, 'WISE_FROM_BANK_TRANSFER': 3.3372533333333334, 'WISE_FROM_BALANCE': 3.3365566666666666}


In [5]:
import pandas as pd

df = pd.DataFrame({k: [v] for k, v in ratios.items()})
df

,utc_time,MASTERCARD,WISE_FROM_BANK_TRANSFER,WISE_FROM_BALANCE
0,2025-06-27 17:18:25,3.334906,3.337253,3.336557


In [6]:
from os import path
outputpath = '../data/rates.csv'

if path.exists(outputpath):
    df.to_csv(outputpath, mode='a', header=False, index=False)
    print('Appended!')
else:
    df.to_csv(outputpath, mode='w', header=True, index=False)
    print('Saved to new file!')

Appended!
